In [94]:
import numpy as np
from scipy import sparse
from scipy.sparse import eye as speye
from scipy.sparse.linalg import cg

from matplotlib import pyplot as plt

import networkx as nx

In [70]:
def compute_d(lamda, eigvals, H, M, mu_norm):
    
    q = 2 * H * (3 * M + 1) ** 2 * mu_norm ** 2 
    v = q / np.log(1 + q / lamda) / eigvals
    
    return np.sum(np.arange(len(v)) <= v)

In [156]:
n = 1000
lamda = 1e-3
tau = 0
sigma = 2
epsilon = 0.01
alpha = 1

gamma_0 = 10

gammas = []

for k in range(100):

    if (k + 1) % 10 == 0:
        print(k)
    else:
        print('.', end='')
        
    p_inner = np.log(n // 2) / (n // 2)
    p_cross = p_inner / np.sqrt(n // 2)
    graph = nx.stochastic_block_model([n // 2, n - n // 2], [[p_inner, p_cross], [p_cross, p_inner]])

    L = nx.linalg.laplacian_matrix(graph) + lamda * sparse.eye(n)

    mu = np.ones(n)
    mu[n // 2:] = -1

    H = np.sum(1 / (abs(mu - tau) + epsilon)**2)
    mu_norm = np.sqrt((mu - tau) @ (L @ (mu - tau)))

    eigvals = np.sort(np.linalg.eigvals(L.A))
    gamma = gamma_0

    for i in range(100):
        M = max(np.sqrt(1 + alpha), np.sqrt(alpha / gamma / lamda))
        d = compute_d(lamda, eigvals, H, M, mu_norm)
        gamma = sigma / mu_norm * np.sqrt(2 * d * np.log(1 + 2 * H * (3 * M + 1) ** 2 * mu_norm ** 2 / lamda))

    gammas.append(gamma)

print(np.mean(gammas), np.std(gammas))

.........9
.........19
.........29
.........39
.........49
.........59
.........69
.........79
.........89
.........99
19.6563297571 0.793702319609


In [162]:
n = 1000
m = 4
p = 0.01
lamda = 1e-3
lamda_gen = n
tau = 0.5
sigma = 1 / 2
epsilon = 0.01
alpha = 1e-8

gamma_0 = 10

gammas = []

for k in range(100):

    if (k + 1) % 10 == 0:
        print(k)
    else:
        print('.', end='')
    
    graph = nx.newman_watts_strogatz_graph(n, m, p)
    
    L = nx.linalg.laplacian_matrix(graph) + speye(n) * lamda
    
    # generate random smooth signal and clip to [0, 1]
    mu, info = cg(L + speye(n) / lamda_gen / n, np.random.randn(n))
    mu -= np.median(mu)
    mu /= np.std(mu) * 5
    mu += 0.5
    mu = np.clip(mu, 0, 1)
        
    H = np.sum(1 / (abs(mu - tau) + epsilon)**2)
    mu_norm = np.sqrt((mu - tau) @ (L @ (mu - tau)))

    eigvals = np.sort(np.linalg.eigvals(L.A))
    gamma = gamma_0

    for i in range(100):
        M = max(np.sqrt(1 + alpha), np.sqrt(alpha / gamma / lamda))
        d = compute_d(lamda, eigvals, H, M, mu_norm)
        gamma = sigma / mu_norm * np.sqrt(2 * d * np.log(1 + 2 * H * (3 * M + 1) ** 2 * mu_norm ** 2 / lamda))

    gammas.append(gamma)

print(np.mean(gammas), np.std(gammas))

.........9
.........19
.........29
.........39
.........49
.........59
.........69
.........79
.........89
.........99
163.00066577 33.4991047388


In [159]:
n = 1000
D = 20
lamda = 1e-5
tau = 0
epsilon = 0.1
alpha = 1e-8
sigma = 1 / 2

gamma_0 = 10

graph = nx.disjoint_union_all([nx.complete_graph(n // D) for _ in range(D)])

L = nx.linalg.laplacian_matrix(graph) + lamda * speye(n)

# generate random signal that is constant on cliques
mu = np.repeat(np.random.rand(D) > 0.5, n//D).astype(float) * 2 - 1

H = np.sum(1 / (abs(mu - tau) + epsilon)**2) 
mu_norm = np.sqrt((mu - tau) @ (L @ (mu - tau)))

eigvals = np.sort(np.linalg.eigvalsh(L.A))
gamma = gamma_0

for i in range(10):
    M = max(np.sqrt(1 + alpha), np.sqrt(alpha / gamma / lamda))
    d = compute_d(lamda, eigvals, H, M, mu_norm)
    print(gamma)
    gamma = sigma / mu_norm * np.sqrt(2 * d * np.log(1 + 2 * H * (3 * M + 1) ** 2 * mu_norm ** 2 / lamda))

T = gamma * H * (3 * M + 1) ** 2 * mu_norm ** 2

print(d, gamma, M, T)

10
130.731121549
130.731121549
130.731121549
130.731121549
130.731121549
130.731121549
130.731121549
130.731121549
130.731121549
20 130.731121549 1.000000005 17286.7600093
